#**Tratamento da Base de dados**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Leitura

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
pip install pycm

In [ ]:
from pycm import *

In [ ]:
falhas = pd.read_csv('/kaggle/input/faulty-steel-plates/faults.csv')
falhas.head()

In [ ]:
falhas.columns

In [ ]:
len(falhas.columns)

In [ ]:
X = falhas.iloc[:,0:27]
Y = falhas.iloc[:,27:34]

In [ ]:
X.columns

In [ ]:
Y.columns

### Padronização
MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range =(0,1))

X_std = pd.DataFrame(sc.fit_transform(X), columns = X.columns)
print(X_std.shape)
X_std.boxplot(figsize = (12.8,8), rot = 90)
plt.boxplot
plt.show()

## Avaliação da base de dados

In [ ]:
X.isna().sum()

In [ ]:
Y.isna().sum()

# PCA

In [ ]:
from sklearn.decomposition import PCA
# Aplicação do Método PCA
pca = PCA()
pca.fit(X_std) # ajuste do método para base padronizada

In [ ]:
print(pca.explained_variance_ratio_) # % da variância explicada por componentes

In [ ]:
PC_values = np.arange(pca.n_components_) + 1
plt.figure(figsize=(12.8,8))
plt.plot(PC_values, pca.explained_variance_ratio_, 'ro-', linewidth=1)
plt.title('Scree Plot - Base original',  fontsize=18)
plt.xlabel('Componente Principal', fontsize=18)
plt.ylabel('Variância Explicada',  fontsize=18)

In [ ]:
PC_values = np.arange(pca.n_components_) + 1
df5 = pd.DataFrame({'PCs':PC_values[0:10], 'Variancia_exp':pca.explained_variance_ratio_[0:10]})
df5['Variancia_exp_somada'] = df5['Variancia_exp'].cumsum()
print(df5)

In [ ]:
# 10 variáveis já são suficientes para explicar 95,48% da variância acumulada da base total.

In [ ]:
X_PCA = PCA(n_components = 10).fit_transform(X_std)

# MODELO RNA

## Divisão da base de dados

In [ ]:
from sklearn.model_selection import train_test_split

xTrain, xTest, yTrain, yTest = train_test_split(X_std, Y, test_size = 0.3, random_state = 0)
xTrainPCA, xTestPCA, yTrainPCA, yTestPCA = train_test_split(X_PCA, Y, test_size = 0.3, random_state = 0)

#verificação

print('===============================================================================')
print('============================Normalizada========================================')

print(xTrain.shape)
print(yTest.shape)
print('\n')
print('===============================================================================')
print('============================PCA================================================')

print(xTrainPCA.shape)
print(yTestPCA.shape)

## MLP

Base normalizada

In [ ]:
#Bibliotecas de aprendizado de máquina
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense

### Treinando o modelo

In [ ]:
model = Sequential()

model.add(Dense(256, input_dim = 27, activation='relu')),
model.add(Dense(128, activation='relu')),
model.add(Dense(64, activation='relu')),
model.add(Dense(7, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(xTrain, yTrain, epochs=100, validation_data=(xTest, yTest), verbose = 0)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

### Acurácia do modelo com a base normal

In [ ]:
# Avaliando o desempenho da rede neural MLP
desempenho = model.evaluate(xTest , yTest , verbose =0)
print(" Acuracia : %.2f%%"%(desempenho[1]*100))

### Matriz de confusão

In [ ]:
predictions1 = model.predict(xTest)
predictions1 = pd.DataFrame(predictions1, columns = yTest.columns)

In [ ]:
true_class = np.argmax(yTest.to_numpy(), axis =1)
pred_class = np.argmax(predictions1.to_numpy(), axis=1)

In [ ]:
def organiza_CM(true_class, pred_class, yTest):
  t_clase = list()
  p_clase = list()
  
  for i in range(len(true_class)):
    for j in range(len(yTest.columns)):
      if true_class[i] == yTest.columns.get_loc(yTest.columns[j]):
          t_clase.append(yTest.columns[j])
      else:
        pass

  for i in range(len(pred_class)):
    for j in range(len(yTest.columns)):
      if pred_class[i] == yTest.columns.get_loc(yTest.columns[j]):
          p_clase.append(yTest.columns[j])
      else:
        pass
  return t_clase, p_clase

In [ ]:
t_clase, p_clase = organiza_CM(true_class, pred_class, yTest)

In [ ]:
cm1 = ConfusionMatrix(actual_vector=t_clase, predict_vector=p_clase)
cm1.plot(cmap=plt.cm.Greens, number_label=True, plot_lib="matplotlib")
plt.xticks(rotation=90)

In [ ]:
#Acurácia por tipo de defeito
cm1.ACC

## PCA - Aplicação da técnica de redução de dimensão

### Treinando o modelo

In [ ]:
model2 = Sequential()
model2.add(Dense(16, input_dim = 10, activation='relu')),
model2.add(Dense(16, activation='relu')),
model2.add(Dense(7, activation='softmax'))
model2.summary()

In [ ]:
model2.compile(optimizer='Adamax',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history2 = model2.fit(xTrainPCA, yTrainPCA, epochs=250, validation_data=(xTestPCA, yTestPCA), verbose = 0)

In [ ]:
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Loss do modelo')
plt.ylabel('Loss')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

### Acurácia do modelo com PCA

In [ ]:
# Avaliando o desempenho da rede neural MLP Base de dados PCA
desempenho2 = model2.evaluate(xTestPCA , yTestPCA , verbose =0)
print(" Acuracia : %.2f%%"%(desempenho2[1]*100))

### Matriz de confusão

In [ ]:
predictions = model2.predict(xTestPCA)
predictions = pd.DataFrame(predictions, columns = yTestPCA.columns)

In [ ]:
true_class = np.argmax(yTestPCA.to_numpy(), axis =1)
pred_class = np.argmax(predictions.to_numpy(), axis=1)

In [ ]:
t_clase_pca, p_clase_pca = organiza_CM(true_class, pred_class, yTestPCA)

In [ ]:
cmpca = ConfusionMatrix(actual_vector=t_clase_pca, predict_vector=p_clase_pca)
cmpca.plot(cmap=plt.cm.Greens, number_label=True, plot_lib="matplotlib")
plt.xticks(rotation=90)

### Acurácia por tipo de defeito

In [ ]:
#Acurácia por tipo de defeito para a rede treinada com a base reduzida por PCA
cmpca.ACC

# BALANCEAMENTO DAS CLASSES



### Como as classes estão balanceadas

In [ ]:
#Base normal
fPastry    = Y.Pastry.sum()/Y.shape[0]
fZ_Scratch = Y.Z_Scratch.sum()/Y.shape[0]
fK_Scatch    = Y.K_Scatch.sum()/Y.shape[0]
fStain    = Y.Stains.sum()/Y.shape[0]
fDirtiness    = Y.Dirtiness.sum()/Y.shape[0]
fBumps = Y.Bumps.sum()/Y.shape[0]
fOther_Faults = Y.Other_Faults.sum()/Y.shape[0]

#PCA
fPastrypca    = yTestPCA.Pastry.sum()/yTestPCA.shape[0]
fZ_Scratchpca = yTestPCA.Z_Scratch.sum()/yTestPCA.shape[0]
fK_Scatchpca    = yTestPCA.K_Scatch.sum()/yTestPCA.shape[0]
fStainpca    = yTestPCA.Stains.sum()/yTestPCA.shape[0]
fDirtinesspca    = yTestPCA.Dirtiness.sum()/yTestPCA.shape[0]
fBumpspca = yTestPCA.Bumps.sum()/yTestPCA.shape[0]
fOther_Faultspca = yTestPCA.Other_Faults.sum()/yTestPCA.shape[0]


In [ ]:
### BASE NORMAL

print('Balanceamento por tipo de defeito para a base normal\n')
print('Pastry: %.2f%%'%(100*fPastry))
print('Z_Scratch: %.2f%%'%(100*fZ_Scratch))
print('K_Scatch: %.2f%%'%(100*fK_Scatch))
print('Stain: %.2f%%'%(100*fStain))
print('Dirtiness: %.2f%%'%(100*fDirtiness))
print('Bumps: %.2f%%'%(100*fBumps))
print('Other Faults: %.2f%%'%(100*fOther_Faults))

print('\n\nBalanceamento por tipo de defeito para a base PCA\n')
print('Pastry: %.2f%%'%(100*fPastrypca))
print('Z_Scratch: %.2f%%'%(100*fZ_Scratchpca))
print('K_Scatch: %.2f%%'%(100*fK_Scatchpca))
print('Stain: %.2f%%'%(100*fStainpca))
print('Dirtiness: %.2f%%'%(100*fDirtinesspca))
print('Bumps: %.2f%%'%(100*fBumpspca))
print('Other Faults: %.2f%%'%(100*fOther_Faultspca))

## Técnica SMOTE - Imputação sintética

Dado que os dados não tem balanceamento por tipo de defeito, vamos usar a técnica de imputação sintética (SMOTE)

In [ ]:
from imblearn.combine import SMOTETomek

In [ ]:
df2 = falhas.iloc[:,27:34].mask(falhas.iloc[:,27:34].eq(1), falhas.columns.to_series(), axis=1)
df2[df2 == 0] = ''
df3 = df2['Pastry'] + df2['Z_Scratch'] + df2['K_Scatch'] + df2['Stains'] + df2['Dirtiness'] + df2['Bumps'] + df2['Other_Faults'] 

Fault = X_std
Fault['Fault'] = df3

Fault

In [ ]:
X2 = Fault.drop('Fault', axis = 1)
y2 = Fault['Fault']

y2.head()

In [ ]:
X_resampled, y_resampled = SMOTETomek().fit_resample(X2, y2)

In [ ]:
X_resampled.shape

In [ ]:
y_resampled.shape

In [ ]:
y_resampled = pd.DataFrame(y_resampled)
X_resampled = pd.DataFrame(X_resampled, columns = X2.columns)

In [ ]:
print('Balanceamento das classes após imputação sintética em %\n')
round(100*y_resampled.value_counts() / len(y_resampled),2)

In [ ]:
#Dummie variables - Voltando cada variável para a coluna 
y_res2 = pd.get_dummies(y_resampled)

In [ ]:
y_res2.columns

In [ ]:
#Os nomes aparecem com Fault_, então vamos acertar para ficar igual

In [ ]:
y_res2.columns = ['Bumps', 'Dirtiness', 'K_Scatch', 'Other_Faults', 'Pastry', 'Stains', 'Z_Scratch']

In [ ]:
y_res2[y_res2.columns.to_list()]

In [ ]:
#A ordem está trocada, na hora de montar a matriz de confusão a função organiza_CM deve ter a mesma ordem que a original. Vamos acertar.

In [ ]:
y_res2 = y_res2[Y.columns.to_list()]

In [ ]:
y_res2

### Divisão da base para treino e validação

In [ ]:
#Redividindo a base de dados de treinamento e validação para a base expandida de |SMOTE|
xTrain2, xTest2, yTrain2, yTest2 = train_test_split(X_resampled, y_res2, test_size = 0.3, random_state = 0)

In [ ]:
yTest2.columns

### Treinando o modelo

In [ ]:
model3 = Sequential()
model3.add(Dense(128, input_dim = 27, activation='relu')),
model3.add(Dense(64, activation='relu')),
model3.add(Dense(7, activation='softmax'))
model3.summary()


model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy']) 

In [ ]:
history3 = model3.fit(xTrain2, yTrain2, epochs=200, validation_data=(xTest2, yTest2), verbose = 0)

In [ ]:
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('Loss do modelo')
plt.ylabel('Loss')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

### Acurácia na base com SMOTE

In [ ]:
desempenho3 = model3.evaluate(xTest2 , yTest2 , verbose =0)
print(" Acuracia : %.2f%%"%(desempenho3[1]*100))

### Acurácia na base de teste normal, sem oversampling após treinamento com SMOTE.

In [ ]:
desempenho3 = model3.evaluate(xTest , yTest , verbose =0)
print(" Acuracia : %.2f%%"%(desempenho3[1]*100))

### Matriz de confusão

In [ ]:
predictions3 = model3.predict(xTest)
predictions3 = pd.DataFrame(predictions3, columns = yTest2.columns)

In [ ]:
true_class2 = np.argmax(yTest.to_numpy(), axis =1)
pred_class2 = np.argmax(predictions3.to_numpy(), axis=1)

In [ ]:
t_clase_smote, p_clase_smote = organiza_CM(true_class2, pred_class2, yTest)

In [ ]:
cm_smote = ConfusionMatrix(actual_vector=p_clase_smote, predict_vector=t_clase_smote)
cm_smote.plot(cmap=plt.cm.Greens, number_label=True, plot_lib="matplotlib")
plt.xticks(rotation=90)

### Acurácia por tipo de defeito

In [ ]:
cm_smote.ACC

# HIPERPARÂMETROS

Usando o pacote de tunagem Keras-Tuner

In [ ]:
pip install keras-tuner

In [ ]:
import math
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense

In [ ]:
import keras_tuner as kt

def build_model(hp):
  model = Sequential()
  
  # Tune das camadas
  # Escolha da quantidade de neurônio entre 0-128
  hp_units1 = hp.Int('neurons_camada1', min_value=16, max_value=128, step=16)
  hp_units2 = hp.Int('neurons_camada2', min_value=0, max_value=128, step=16)
  hp_units3 = hp.Int('neurons_camada3', min_value=0, max_value=128, step=16)
  
  ativacao1 =  hp.Choice('ativacao_camada1', values=['relu', 'sigmoid', 'tanh'], default='relu' )
  ativacao2 =  hp.Choice('ativacao_camada2', values=['relu', 'sigmoid', 'tanh'], default='relu' )

  model.add(Dense(units=hp_units1, activation= 'relu'))
  model.add(Dense(units=hp_units2, activation= ativacao1))
  model.add(Dense(units=hp_units3, activation= ativacao2))
  
  #Saída
  model.add(Dense(7, activation='softmax'))

  
  model.compile(
      optimizer= hp.Choice('otimizador', values=['adam', 'adamax', 'nadam'], default='adam' ),
      loss= 'categorical_crossentropy',
      metrics=['accuracy']
  )

  return model

## Base normal

In [ ]:
# HyperBand - keras tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=100,
    directory='keras_tuner_dir',
    project_name='melhor_model'
)

tuner.search(xTrain, yTrain, epochs=100, validation_data=(xTest, yTest))

In [ ]:
#Resumo do melhor modelo encontrado
tuner.results_summary(1)

In [ ]:
melhor_modelo = tuner.get_best_models(num_models=2)

In [ ]:
history_melhor_modelo = melhor_modelo[0].fit(xTrain, yTrain, epochs=100, validation_data=(xTest, yTest), verbose = 0)

In [ ]:
plt.plot(history_melhor_modelo.history['accuracy'])
plt.plot(history_melhor_modelo.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

## Base PCA

In [ ]:
# HyperBand algorithm from keras tuner
tunerpca = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=100,
    directory='keras_tuner_dir',
    project_name='melhor_modelo_PCA'
)

tunerpca.search(xTrainPCA, yTrainPCA, epochs=100, validation_data=(xTestPCA, yTestPCA))

In [ ]:
#Resumo do melhor modelo encontrado
tunerpca.results_summary(1)

In [ ]:
melhor_modelo_pca = tunerpca.get_best_models(num_models=2)

In [ ]:
history_melhor_modelo_pca = melhor_modelo_pca[0].fit(xTrainPCA, yTrainPCA, epochs=100, validation_data=(xTestPCA, yTestPCA), verbose = 0)

In [ ]:
plt.plot(history_melhor_modelo_pca.history['accuracy'])
plt.plot(history_melhor_modelo_pca.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

## Base SMOTE

In [ ]:
# HyperBand algorithm from keras tuner
tunersmote = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=100,
    directory='keras_tuner_dir',
    project_name='melhor_modelo_smote'
)

tunersmote.search(xTrain2, yTrain2, epochs=100, validation_data=(xTest2, yTest2))

In [ ]:
#Resumo do melhor modelo encontrado
tunersmote.results_summary(1)

In [ ]:
melhor_modelo_smote = tunersmote.get_best_models(num_models=2)

In [ ]:
history_melhor_modelo_smote = melhor_modelo_smote[0].fit(xTrain2, yTrain2, epochs=250, validation_data=(xTest2, yTest2), verbose = 0)

In [ ]:
plt.plot(history_melhor_modelo_smote.history['accuracy'])
plt.plot(history_melhor_modelo_smote.history['val_accuracy'])
plt.title('Acurácia do modelo')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['treinamento', 'validação'], loc='upper left')
plt.show()

In [ ]:
tunersmote.search_space_summary(extended=False)

In [ ]:
desempenho4 = melhor_modelo_smote[0].evaluate(xTest2 , yTest2 , verbose =0)
print(" Acuracia : %.2f%%"%(desempenho4[1]*100))

In [ ]:
desempenho5 = melhor_modelo_smote[0].evaluate(xTest , yTest , verbose =0)
print(" Acuracia : %.2f%%"%(desempenho5[1]*100))

### Matriz de confusão

In [ ]:
predictions5 = melhor_modelo_smote[0].predict(xTest)
predictions5 = pd.DataFrame(predictions5, columns = yTest2.columns)

In [ ]:
true_class = np.argmax(yTest.to_numpy(), axis =1)
pred_class = np.argmax(predictions5.to_numpy(), axis=1)

In [ ]:
t_clase_smote, p_clase_smote = organiza_CM(true_class, pred_class, yTest)

In [ ]:
cm_smote = ConfusionMatrix(actual_vector=p_clase_smote, predict_vector=t_clase_smote)
cm_smote.plot(cmap=plt.cm.Greens, number_label=True, plot_lib="matplotlib")
plt.xticks(rotation=90)

# Verificação do desempenho médio

### Base normal

In [ ]:
def cross_val_teste(X, y, modelo):
  X_std = pd.DataFrame(sc.fit_transform(X), columns = X.columns)
  Y = y
  
  xTrain, xTest, yTrain, yTest = train_test_split(X_std, Y, test_size = 0.3, random_state = 0)
  
  modelo[0].fit(xTrain, yTrain, epochs=100, validation_data=(xTest, yTest), verbose = 0)
  acuracia = modelo[0].evaluate(xTest , yTest , verbose =0)

  return acuracia[1]

In [ ]:
Desempenho = []

for i in range(0,10):
  Desempenho.append(cross_val_teste(X, Y, melhor_modelo_smote))

In [ ]:
media_acuracia = np.mean(Desempenho)
desvpad_acuracia = np.std(Desempenho)

In [ ]:
print('Média da acurácia do melhor modelo SMOTE: %.2f%%'%(100*media_acuracia))
print('Desvio padrão da acurácia do melhor modelo SMOTE: %.2f%%'%(100*desvpad_acuracia))

### Base reduzida via PCA

In [ ]:
def cross_val_teste_PCA(X, y, modelo):
  X_std = pd.DataFrame(sc.fit_transform(X), columns = X.columns)
  X_PCA = PCA(n_components = 10).fit_transform(X_std)
  Y = y
  
  xTrain, xTest, yTrain, yTest = train_test_split(X_PCA, Y, test_size = 0.3, random_state = 0)
  
  modelo[0].fit(xTrain, yTrain, epochs=100, validation_data=(xTest, yTest), verbose = 0)
  acuracia = modelo[0].evaluate(xTest , yTest , verbose =0)

  return acuracia[1]

In [ ]:
Desempenhopca = []

for i in range(0,10):
  Desempenhopca.append(cross_val_teste_PCA(X, Y, melhor_modelo_pca))

In [ ]:
media_acuracia_pca = np.mean(Desempenhopca)
desvpad_acuracia_pca = np.std(Desempenhopca)

In [ ]:
print('Média da acurácia do melhor modelo PCA: %.2f%%'%(100*media_acuracia_pca))
print('desvio padrão da acurácia do melhor modelo PCA: %.2f%%'%(100*desvpad_acuracia_pca))

### Avaliando a diferença de desempenho por teste-t
Entre o modelo smote e o modelo PCA

In [ ]:
from scipy import stats

In [ ]:
stats.ttest_ind(Desempenho, Desempenhopca)

In [ ]:
#Pelo valor p menor que 0,05, há diferenças entre os valores dos modelos. 
# Visualizando via gráfico de boxplot

In [ ]:
desempenhos = [Desempenho, Desempenhopca]

plt.boxplot(desempenhos)
plt.xticks([1,2], ['SMOTE', 'PCA'])
plt.show()

# Desempenho final - via matriz de confusão

In [ ]:
print('Desempenho via PCA', round(100*cmpca.Overall_ACC,2), '%')

In [ ]:
print('Desempenho via SMOTE', round(100*cm_smote.Overall_ACC,2), '%')